#Extrovert vs. Introvert Behavior Data

## Problem statement

## Data collection

## Exploration and data cleaning

## Analysis of univariate variables

### categorical variables

### numeric variables

## Analysis of multivariate variables

## Numerical-numerical analysis


### categorical-categorical analysis


#### TODO: deterine whether data needs this section


### Correlation analysis


#### skipped; covered by heatmap in a cell not much further below


#### numerical-categorical analysis


## Building decision tree mcdel


### Splitting training vs test data


### Scaling


### feature selection for selecting most-relevant features to use for building ML model


### Accuracy score without hyperparameter optimization


## Tuning


### Hyperparameter optimizing

#### Grid search


#### Random search hyperparameter optimization-hunting


## Saving the model